In [1]:
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pygetwindow as gw
import pyautogui
import pandas as pd
import copy
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# 항상 같은 위치에서 열린다.
# 드래그로 원하는 위치를 잡아줘야할듯

url = 'https://map.kakao.com/'
# driver = webdriver.Chrome('./chromedriver',chrome_options=options) # 크롬창 숨기기
options = webdriver.ChromeOptions()
driver = webdriver.Chrome(options=options)

driver.get(url)

# 윈도우창 위치, 크기 조정
chrome = gw.getWindowsWithTitle('카카오맵')[0]

chrome.moveTo(0, 0)
chrome.resizeTo(1395, 1000)

# 교육장을 검색해서 가운데로 위치하게 하자
# 창이 열리고 바로 검색을 하면 검색 버튼을 찾지 못한다.
# 때문에 모션을 실행 후 검색

pyautogui.PAUSE = 0.5
pyautogui.click(750,750)
pyautogui.click(750,750)
pyautogui.press('+')

search = driver.find_element(By.CSS_SELECTOR, '#search\.keyword\.query')
search.clear()
search.send_keys('청년취업사관학교 영등포캠퍼스')

sch_btn = driver.find_element(By.CSS_SELECTOR, 'div.box_searchbar > button')
sch_btn.click()

pyautogui.click(750,750)
pyautogui.press('+')

# 현 지도 내 장소검색 클릭
sch_btn = driver.find_element(By.CSS_SELECTOR, '#search\.keyword\.bounds')
sch_btn.click()

drag_position = [[[1000,600],[1610,1130]],[[1225,600],[0,600]],[[800,950],[800,-100]],[[420,600],[1645,600]]]

res_lst = pd.DataFrame(columns=['가맹점명','주소','분류','점수','건수'])
res_lst

# chrome = gw.getWindowsWithTitle('카카오맵')[0] 

# pyautogui.doubleClick(chrome.left, chrome.top)
cnt = 0

for lab, po_lst in enumerate(drag_position):
    print(po_lst)
    pyautogui.moveTo(po_lst[0])
    pyautogui.dragTo(po_lst[1],duration=0.25)
    pyautogui.sleep(0.5)

    print(lab)  
    if lab == 0:
        search = driver.find_element(By.CSS_SELECTOR, '#search\.keyword\.query')
        search.clear()
        search.send_keys('식당')
        sch_btn = driver.find_element(By.CSS_SELECTOR, 'div.box_searchbar > button')
        sch_btn.click()
        # pyautogui.click(363,291)
        pyautogui.sleep(0.5)
        search = driver.find_element(By.CSS_SELECTOR, '#info\.search\.place\.more')
        search.click()
        pyautogui.sleep(0.5)

    off = 0
    while off == 0:
        print("while 시작")
        
        for page in range(1,6):
    
            print('-'*10,f'{page} page','-'*10)
            
            try:
                search = driver.find_element(By.CSS_SELECTOR, f'#info\.search\.page\.no{page}')
                search.click()
                pyautogui.sleep(0.5)
            except:
                off = 1
                break
        
            for num in range(1,16):
                search_name = driver.find_element(By.CSS_SELECTOR, '#info\.noPlace > div')
                
                if search_name.text == '':
                    try:
                        search_name = driver.find_element(By.CSS_SELECTOR, f'li:nth-child({num}) > div.head_item.clickArea > strong > a.link_name')
                        # print(num,search_name.text)
                        res_lst.loc[cnt,"가맹점명"] = search_name.text
                    except:
                        # print('error')
                        res_lst.loc[cnt,"가맹점명"] = 'error'

                    try:
                        search_add = driver.find_element(By.CSS_SELECTOR, f'li:nth-child({num}) > div.info_item > div.addr > p:nth-child(1)')
                        res_lst.loc[cnt,"주소"] = search_add.text
                    except:
                        res_lst.loc[cnt,"주소"] = 'error'

                    try:
                        search_add = driver.find_element(By.CSS_SELECTOR, f'li:nth-child({num}) > div.head_item.clickArea > span')
                        res_lst.loc[cnt,"분류"] = search_add.text
                    except:
                        res_lst.loc[cnt,"분류"] = 'error'

                    try:
                        star = driver.find_element(By.CSS_SELECTOR, f'li:nth-child({num}) > div.rating.clickArea > span.score > em')
                        no_review = driver.find_element(By.CSS_SELECTOR, f'li:nth-child({num}) > div.rating.clickArea > span.txt_blind')
                        if no_review.text == "" and star.text != "":
                            res_lst.loc[cnt,"점수"] = star.text
                        elif no_review.text != "" and star.text == "":
                            res_lst.loc[cnt,"점수"] = no_review.text
                    except:
                        res_lst.loc[cnt,"점수"] = 'error'

                    try:
                        num = driver.find_element(By.CSS_SELECTOR, f'li:nth-child({num}) > div.rating.clickArea > span.score > a')
                        res_lst.loc[cnt,"건수"] = num.text.replace('건','')
                    except:
                        res_lst.loc[cnt,"건수"] = 'error'

                cnt += 1

            if page == 5:
                search = driver.find_element(By.CSS_SELECTOR, '#info\.search\.page\.next')
                if search.get_attribute("class") == 'next':
                    search.click()
                    pyautogui.sleep(0.5)
                elif search.get_attribute("class") == 'next disabled':
                    off = 1

driver.quit()
                    
# <button type="button" id="info.search.page.next" class="next">다음</button>
# <button type="button" id="info.search.page.next" class="next disabled">다음</button>

[[1000, 600], [1610, 1130]]
0
while 시작
---------- 1 page ----------
---------- 2 page ----------
---------- 3 page ----------
---------- 4 page ----------
---------- 5 page ----------
while 시작
---------- 1 page ----------
---------- 2 page ----------
[[1225, 600], [0, 600]]
1
while 시작
---------- 1 page ----------
---------- 2 page ----------
---------- 3 page ----------
---------- 4 page ----------
---------- 5 page ----------
while 시작
---------- 1 page ----------
---------- 2 page ----------
---------- 3 page ----------
---------- 4 page ----------
---------- 5 page ----------
while 시작
---------- 1 page ----------
---------- 2 page ----------
---------- 3 page ----------
---------- 4 page ----------
---------- 5 page ----------
while 시작
---------- 1 page ----------
---------- 2 page ----------
[[800, 950], [800, -100]]
2
while 시작
---------- 1 page ----------
---------- 2 page ----------
---------- 3 page ----------
---------- 4 page ----------
---------- 5 page ----------
while 시작
---

In [12]:
# 겹치는 지도로 인한 중복 식당 제거
res_lst.drop_duplicates(inplace=True)
res_lst

,가맹점명,주소,분류,점수,건수
0,화사랑화로구이 문래점,"서울 영등포구 선유로9가길 16 광양림빌딩 1,2층","육류,고기",3.5,27
1,라메흐꺄도,서울 영등포구 선유로17길 24 신일아르디세 1층 105~106호,"해물,생선",4.1,43
2,화덕으로간고등어반상,서울 영등포구 선유로9길 10 문래SKV1센터 1층 119호,"해물,생선",4.5,2
3,쁘띠문래,서울 영등포구 선유로17길 24 신일아르디세 1층 109호,양식,4.1,23
4,문래옥,서울 영등포구 선유로17길 24 1층,"육류,고기",4.1,16
...,...,...,...,...,...
522,설봄,서울 영등포구 선유서로 17 1층 101호,"떡,한과",4.3,10
528,김가네 문래SKV1점,서울 영등포구 선유로9길 10 1층 108호,분식,2.3,3
529,라라커피,서울 영등포구 선유로9나길 12 문래 두산위브 상가동 1층 105호,카페,4.8,4
530,피자알볼로 문래점,서울 영등포구 선유로3길 10 하우스디비즈 1층 101호,피자,3.8,4


In [13]:
res_lst.reset_index(drop=True,inplace=True)
res_lst

,가맹점명,주소,분류,점수,건수
0,화사랑화로구이 문래점,"서울 영등포구 선유로9가길 16 광양림빌딩 1,2층","육류,고기",3.5,27
1,라메흐꺄도,서울 영등포구 선유로17길 24 신일아르디세 1층 105~106호,"해물,생선",4.1,43
2,화덕으로간고등어반상,서울 영등포구 선유로9길 10 문래SKV1센터 1층 119호,"해물,생선",4.5,2
3,쁘띠문래,서울 영등포구 선유로17길 24 신일아르디세 1층 109호,양식,4.1,23
4,문래옥,서울 영등포구 선유로17길 24 1층,"육류,고기",4.1,16
...,...,...,...,...,...
386,설봄,서울 영등포구 선유서로 17 1층 101호,"떡,한과",4.3,10
387,김가네 문래SKV1점,서울 영등포구 선유로9길 10 1층 108호,분식,2.3,3
388,라라커피,서울 영등포구 선유로9나길 12 문래 두산위브 상가동 1층 105호,카페,4.8,4
389,피자알볼로 문래점,서울 영등포구 선유로3길 10 하우스디비즈 1층 101호,피자,3.8,4


In [14]:
res_lst

,가맹점명,주소,분류,점수,건수
0,화사랑화로구이 문래점,"서울 영등포구 선유로9가길 16 광양림빌딩 1,2층","육류,고기",3.5,27
1,라메흐꺄도,서울 영등포구 선유로17길 24 신일아르디세 1층 105~106호,"해물,생선",4.1,43
2,화덕으로간고등어반상,서울 영등포구 선유로9길 10 문래SKV1센터 1층 119호,"해물,생선",4.5,2
3,쁘띠문래,서울 영등포구 선유로17길 24 신일아르디세 1층 109호,양식,4.1,23
4,문래옥,서울 영등포구 선유로17길 24 1층,"육류,고기",4.1,16
...,...,...,...,...,...
386,설봄,서울 영등포구 선유서로 17 1층 101호,"떡,한과",4.3,10
387,김가네 문래SKV1점,서울 영등포구 선유로9길 10 1층 108호,분식,2.3,3
388,라라커피,서울 영등포구 선유로9나길 12 문래 두산위브 상가동 1층 105호,카페,4.8,4
389,피자알볼로 문래점,서울 영등포구 선유로3길 10 하우스디비즈 1층 101호,피자,3.8,4


In [15]:
res_lst_drop = res_lst.drop(res_lst.loc[res_lst['가맹점명']=='error'].index)
res_lst_drop.reset_index(drop=True,inplace=True)
res_lst_drop

,가맹점명,주소,분류,점수,건수
0,화사랑화로구이 문래점,"서울 영등포구 선유로9가길 16 광양림빌딩 1,2층","육류,고기",3.5,27
1,라메흐꺄도,서울 영등포구 선유로17길 24 신일아르디세 1층 105~106호,"해물,생선",4.1,43
2,화덕으로간고등어반상,서울 영등포구 선유로9길 10 문래SKV1센터 1층 119호,"해물,생선",4.5,2
3,쁘띠문래,서울 영등포구 선유로17길 24 신일아르디세 1층 109호,양식,4.1,23
4,문래옥,서울 영등포구 선유로17길 24 1층,"육류,고기",4.1,16
...,...,...,...,...,...
385,설봄,서울 영등포구 선유서로 17 1층 101호,"떡,한과",4.3,10
386,김가네 문래SKV1점,서울 영등포구 선유로9길 10 1층 108호,분식,2.3,3
387,라라커피,서울 영등포구 선유로9나길 12 문래 두산위브 상가동 1층 105호,카페,4.8,4
388,피자알볼로 문래점,서울 영등포구 선유로3길 10 하우스디비즈 1층 101호,피자,3.8,4


In [16]:
res_lst.to_excel("주변식당목록_error.xlsx",index=False)
res_lst_drop.to_excel("주변식당목록.xlsx",index=False)